# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
Arxiv has 78 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.14260


extracting tarball to tmp_2508.14260... done.


N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\hsize]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[angle=-90,width=3cm]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_d

Found 170 bibliographic references in tmp_2508.14260/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2508.14397


extracting tarball to tmp_2508.14397... done.
Retrieving document from  https://arxiv.org/e-print/2508.14665


extracting tarball to tmp_2508.14665... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.14260-b31b1b.svg)](https://arxiv.org/abs/2508.14260) | **Efficient black hole seed formation in low metallicity and dense stellar clusters with implications for JWST sources**  |
|| M. Vergara, et al. -- incl., <mark>N. Neumayer</mark> |
|*Appeared on*| *2025-08-21*|
|*Comments*| *16 pages, 12 figures*|
|**Abstract**|            Recent observations with the James Webb Space Telescope (JWST) reveal young massive clusters (YMCs) as key building blocks of early galaxies. They are not only important constituents of galaxies, but also potential birthplaces of very massive stars (VMSs) and black hole (BH) seeds. We explore stellar dynamics in extremely dense clusters with initial half-mass densities of $\rho_h \gtrsim 10^8M_\odot{\rm pc}^{-3}$ at very low metallicity, comparable to some of the densest clusters seen by JWST. Using direct N-body and Monte Carlo simulations with stellar evolution, we show that VMS formation through collisions is unavoidable, with final masses reaching $5\times10^3$ to $4\times10^4M_\odot$. These results support the existence of a critical mass scale above which collisions become highly efficient, enabling the formation of VMSs and intermediate-mass BHs (IMBHs). Our models, using nbody6++gpu and MOCCA with updated SSE/BSE routines, show that dense clusters rapidly form VMSs via stellar bombardment. The VMSs then collapse into BH seeds of a few $10^3$ to $10^4M_\odot$ in less than 4 Myr. We identify a critical mass-density threshold beyond which clusters undergo runaway collisions that yield massive BH seeds. For typical YMCs detected by JWST, efficiencies up to 10% are expected, implying BH masses up to $10^5M_\odot$ if formed via collisions. We predict a scaling relation for BH mass, $\log(M_{\rm BH}/M_\odot)=-0.76+0.76\log(M/M_\odot)$. Frequent VMS formation may also explain the high nitrogen abundance observed in galaxies at high redshift.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.14397-b31b1b.svg)](https://arxiv.org/abs/2508.14397) | **Constraining Common Envelope Evolution in Binary Neutron Star Formation with Combined Galactic and Gravitational-Wave Observations**  |
|| Z. Chen, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-08-21*|
|*Comments*| *6 pages, 2 figures, 2 tables; Submitted to MNRAS*|
|**Abstract**|            Binary neutron stars (BNSs) are among the most interesting sources for multimessenger studies. A number of recently discovered BNSs in the Milky Way by radio telescopes have added new information to the parameter distribution of the Galactic BNSs. The scarce of BNS mergers during the O4 run of the LIGO-Virgo-Kagra (LVK) suggests a BNS local merger rate six times lower than the previous constraint obtained by O1-O3 runs. With these new multimessenger observations, in this letter, we adopt the compact binary population synthesis model and Bayesian analysis to constrain the formation and evolution of BNSs, especially the common envelope (CE) evolution. We find that it is required: (1) a fraction ($f_{\rm HG}\sim0.8$) but not all of the Hertzsprung gap donors merged with their companions in the CE stage, in order to simultaneously explain the low BNS merger rate density and the existence of the short-orbital-period ($\lesssim 1$ day) Galactic BNSs, different from either all ($f_{\rm HG}=1$) or none ($f_{\rm HG}=0$) adopted in previous studies; (2) a large CE ejection efficiency $\alpha$ ($\sim 5$), in order to explain the existence of the long-orbital-period ($\gtrsim 10$ day) Galactic BNSs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.14665-b31b1b.svg)](https://arxiv.org/abs/2508.14665) | **Enhancement of first-order phase transitions through a mass-acquiring scalar field**  |
|| Y.-J. Li, <mark>J. Liu</mark>, Z.-K. Guo |
|*Appeared on*| *2025-08-21*|
|*Comments*| *12 pages,10 figures*|
|**Abstract**|            Phase transitions in the early Universe give rise to effective masses for massless fields in the symmetry-broken phase. We use the lattice simulations to investigate the impact of a spectator scalar field with mass generation on the dynamics of first-order phase transitions and the generation of gravitational waves. In addition to the well-known friction effects, we identify a novel effect that significantly enhances the strength of first-order phase transitions. The amplitude of the mass-acquiring field is highly suppressed in the true vacuum bubbles, resulting in additional release of vacuum energy that concentrate on the bubble walls. We also establish an analytical method to explain our numerical results.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.14260/./Plot_Mcrit_relax_fill.png', 'tmp_2508.14260/./histograms_counts_nbody-mocca.png', 'tmp_2508.14260/./cluster__evolution_R0005N50kZ4-R005N750kZ4.png']
copying  tmp_2508.14260/./Plot_Mcrit_relax_fill.png to _build/html/
copying  tmp_2508.14260/./histograms_counts_nbody-mocca.png to _build/html/
copying  tmp_2508.14260/./cluster__evolution_R0005N50kZ4-R005N750kZ4.png to _build/html/
exported in  _build/html/2508.14260.md
    + _build/html/tmp_2508.14260/./Plot_Mcrit_relax_fill.png
    + _build/html/tmp_2508.14260/./histograms_counts_nbody-mocca.png
    + _build/html/tmp_2508.14260/./cluster__evolution_R0005N50kZ4-R005N750kZ4.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ff}[1]{\textcolor{blue}{\bf #1}}$</div>



<div id="title">

# Efficient black hole seed formation in low metallicity and dense stellar clusters   with implications for JWST sources

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.14260-b31b1b.svg)](https://arxiv.org/abs/2508.14260)<mark>Appeared on: 2025-08-21</mark> -  _16 pages, 12 figures_

</div>
<div id="authors">

M. Vergara, et al. -- incl., <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** Recent observations with the James Webb Space Telescope (JWST) have revealed the presence of young massive clusters (YMCs) as building blocks of the first galaxies during the first billion years of the Universe. They are not only important constituents of the galaxies, but also potential birth places of very massive stars (VMSs) and black hole (BH) seeds. In this paper, we explore the stellar dynamics in extremely dense clusters with an initial half-mass density of $\rho_{h} \gtrsim 10^8 \rm M_\odot pc^{-3}$ at very low metallicity. These densities are roughly comparable to some of the densest clusters found by JWST. Our detailed $N$ -body and Monte Carlo simulations, which include stellar evolution, show that the formation of VMSs through collisions is unavoidable, with the resulting final masses reaching $\sim 5 \times 10^3$ to $4 \times 10^4 \rm M_\odot$ . These simulations  serve to verify the hypothesis that there is a critical mass scale at which collisions in the system become very efficient and thus VMSs and potentially intermediate-mass BHs (IMBHs) can form. We use nbody6++gpu and MOCCA , including the latest updates of the stellar evolution routines SSE/BSE, along with specific routines to handle the formation and dynamical evolution of VMSs. We show that dense star clusters rapidly form VMSs due to constant stellar bombardment. The VMSs eventually collapse and form a BH seed with masses ranging from a few $10^{3}$ to a few $10^{4} \rm M_\odot$ in less than $4 $ Myr. We discover a critical mass-density threshold in star clusters, beyond which the latter experience several runaway collisions, leading to the formation of massive BH seeds. Considering the ratio of stellar mass to critical mass for typical YMCs detected via JWST, we expect efficiencies in the range up to $10\%$ for the so far detected clusters, thus corresponding to expected BH masses up to $10^5 \rm M_\odot$ in case of their formation via collisions, we predict a relation for the BH mass that follows the shape of $\log(M_{BH} /\rm M_\odot)=-0.76 + 0.76 \log(M /\rm M_\odot)$ . As a side product, the frequent formation of VMS may naturally explain the high amount of nitrogen found in galaxies at high redshift.

</div>

<div id="div_fig1">

<img src="tmp_2508.14260/./Plot_Mcrit_relax_fill.png" alt="Fig6" width="100%"/>

**Figure 6. -** The dotted lines represent when the timescales ($t_{coll}$ and t$_{rx}$) are equal to $\tau = 4 $Myr. The shaded regions indicate the parameter space where the respective timescales fall within the range $1 $Myr $\leq \tau \leq 10 $Gyr, highlighting different dynamical regimes: pink for two-body relaxation and grey for stellar collisions. Our clusters \texttt{R0005N50k}, \texttt{R001N100k}, \texttt{R001N250k}, \texttt{R005N500k}, and \texttt{R005N750k}, evolve for $3.73$, $3.81$, $3.64$, $3.80$, $3.87 $Myr, respectively. We show their initial conditions with empty symbols and the final conditions with filled symbols, connected by arrows. (*fig_ic*)

</div>
<div id="div_fig2">

<img src="tmp_2508.14260/./histograms_counts_nbody-mocca.png" alt="Fig13" width="100%"/>

**Figure 13. -** Histograms showing the number of collisions contributing to VMS/BH seed formation. The mass bins are: $<0.1 \mathrm{M}_\odot$, $0.1$–$1 \mathrm{M}_\odot$, $1$–$10 \mathrm{M}_\odot$, $10$–$100 \mathrm{M}_\odot$, and $>100 \mathrm{M}_\odot$. Results are shown for both the N-body simulations (solid bars) and the MOCCA simulations (hatched bars). From top to bottom, the models are \texttt{R0005N50k}, \texttt{R001N100k}, \texttt{R001N250k}, \texttt{R005N500k}, and \texttt{R005N750k}. (*fig:hist_coll*)

</div>
<div id="div_fig3">

<img src="tmp_2508.14260/./cluster__evolution_R0005N50kZ4-R005N750kZ4.png" alt="Fig7" width="100%"/>

**Figure 7. -** Lagrangian radii calculated from the initial mass of the cluster for the $90\%$, $70\%$, $50\%$, $30\%$, $10\%$, and $1\%$ of the enclosed mass of model \texttt{R005N750k}(top) and model \texttt{R0005N50k}(bottom). (*R0005N50k-R005N750k*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.14260"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

130  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
